In [1]:
import pandas as pd
import torch

In [3]:
df=pd.read_csv("data/tox21/raw/tox21.csv")
smiles_list=list(df["smiles"])
target = df[['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD',
           'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']]

target = target.replace(0, -1)  # convert 0 to -1
target = target.fillna(0)  

target = torch.tensor(target.values,dtype=torch.float)

In [4]:
target.shape

torch.Size([7831, 12])

In [7]:
target[1].unsqueeze(0).shape

torch.Size([1, 12])

In [2]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem.rdchem import BondType as BT
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from tqdm import tqdm


In [3]:
df=pd.read_csv("data/toxcast/raw/toxcast.csv")
smiles_list=list(df["smiles"])

In [6]:
symbol_list=[]
for i, smile in enumerate(tqdm(df["smiles"])):
    mol = AllChem.MolFromSmiles(smile)
    if mol is None:
        continue
    for atom in mol.GetAtoms():
        symb=atom.GetSymbol()
        if symb not in symbol_list:
            symbol_list.append(symb)

  0%|          | 0/8597 [00:00<?, ?it/s]

100%|██████████| 8597/8597 [00:02<00:00, 3523.49it/s]


In [7]:
strm=""
for i,symb in enumerate(symbol_list):
    st="\'"+symb+"\'"+":"+str(i)+","
    strm+=st

In [8]:
strm

"'O':0,'N':1,'C':2,'Cl':3,'Si':4,'Br':5,'Ba':6,'Nd':7,'Dy':8,'In':9,'P':10,'Sb':11,'Co':12,'S':13,'K':14,'Na':15,'B':16,'Ca':17,'Hg':18,'Ni':19,'Se':20,'Tl':21,'Cd':22,'F':23,'Fe':24,'Li':25,'Yb':26,'I':27,'Cr':28,'Sn':29,'Zn':30,'Cu':31,'Pb':32,'As':33,'Bi':34,'H':35,'Gd':36,'V':37,'Mn':38,'Au':39,'Ti':40,'Zr':41,'Mo':42,'Mg':43,'Eu':44,'Al':45,'Pt':46,'Sr':47,'Sc':48,'Ag':49,'Pd':50,'Be':51,'Ge':52,"